In [ ]:
#Observable trend 1: The data clearly shows that men play much more than women, at a rate of 4 to 1.

In [ ]:
#Observable trend 2:  The normalized data demonstates the other/non-disclosed players have a higher spending price mean than both the 
#women and men categories, but the number of other/non-disclosed is vastly lower than both.

In [ ]:
#Observable trend 3: The age group is dominated by the 20-24 age range by a significant amount meaning the game is primarily
#played by men in their early twenties.

In [4]:
import json

import pandas as pd


In [5]:
df = pd.read_json('purchase_data.json')
df.head(10)

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


In [6]:
#total players
df['SN'].shape

(780,)

In [7]:
#unique players
df['SN'].unique().shape

(573,)

In [8]:
#purchasing analysis
#number of unique items
df['Item Name'].nunique()

179

In [9]:
#avg purchase price
df['Price'].mean()

2.931192307692303

In [10]:
#total # of purchases
df['SN'].count()

780

In [11]:
#total rev
df['Price'].sum()

2286.33

In [12]:
#Gender Demographics
#***count of male, female, and other***
df.groupby(['SN','Gender']).count().reset_index()['Gender'].value_counts()


Male                     465
Female                   100
Other / Non-Disclosed      8
Name: Gender, dtype: int64

In [13]:
#percentage of male, female, and other
df.groupby(['SN','Gender']).count().reset_index()['Gender'].value_counts(normalize=True)*100

Male                     81.151832
Female                   17.452007
Other / Non-Disclosed     1.396161
Name: Gender, dtype: float64

In [14]:
#table of gender demographics
norm_count = df.groupby(['SN','Gender']).count().reset_index()['Gender'].value_counts(normalize=False)
norm_percent = df.groupby(['SN','Gender']).count().reset_index()['Gender'].value_counts(normalize=True)*100
pd.concat([norm_count, norm_percent], axis = 1)

,Gender,Gender
Male,465,81.151832
Female,100,17.452007
Other / Non-Disclosed,8,1.396161


In [15]:
#Purchasing Analysis (Gender)
#The below each broken by gender
#Purchase Count
#Average Purchase Price=total purchase value/purchase ct
#Total Purchase Value
#Normalized Totals=total purchase value/unique users
#grouped = df.groupby('Gender').mean().reset_index()
#grouped

#df.groupby(['SN','Gender']).count().reset_index()['Gender'].value_counts()

purch_gender = df.groupby('Gender').agg(['sum','mean','count'])

level0 = purch_gender.columns.get_level_values(0)
level1 = purch_gender.columns.get_level_values(1)
purch_gender.columns = level0 + '_' + level1
purch_gender = purch_gender[['Price_sum', 'Price_mean', 'Price_count']]
purch_gender

,Price_sum,Price_mean,Price_count
Gender,,,
Female,382.91,2.815515,136
Male,1867.68,2.950521,633
Other / Non-Disclosed,35.74,3.249091,11


In [16]:
#Normalized Totals=total purchase value/unique users
purch_gender = pd.concat([purch_gender, norm_count], axis=1)
purch_gender['Normalized'] = purch_gender.Price_sum / purch_gender.Gender
purch_gender

,Price_sum,Price_mean,Price_count,Gender,Normalized
Female,382.91,2.815515,136,100,3.829100
Male,1867.68,2.950521,633,465,4.016516
Other / Non-Disclosed,35.74,3.249091,11,8,4.467500


In [17]:
#Age Demographics
#Below broken into 4 bins of 4 years
age_bin = df[['Age', 'SN']].drop_duplicates()

ages = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 9999]
a_groups = ["<10", "10-14","15-19","20-24","25-29", "30-34","35-39","40+"]

assert( len(ages)> len(a_groups))

df['Age_Group'] = pd.cut(df['Age'], ages, labels=a_groups)
age_bin['Age_Group'] = pd.cut(age_bin['Age'], ages, labels=a_groups)
age_out = pd.concat([age_bin.Age_Group.value_counts(normalize=True),age_bin.Age_Group.value_counts()],axis=1)
age_out


,Age_Group,Age_Group
20-24,0.452007,259
15-19,0.174520,100
25-29,0.151832,87
30-34,0.082024,47
35-39,0.047120,27
10-14,0.040140,23
<10,0.033159,19
40+,0.019197,11


In [21]:
age_norm = df.groupby('Age_Group').agg(['sum', 'mean','count'])['Price']
age_norm.reset_index(inplace=True)
age_norm['unique_buyers'] = age_norm['Age_Group'].map(lambda x: age_out.to_dict()['Age_Group'].get(x)) 
age_norm['normed_mean'] = age_norm['unique_buyers'].astype('float')
age_norm

C:\Users\Josue\Anaconda3.1\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


,Age_Group,sum,mean,count,unique_buyers,normed_mean
0,<10,83.46,2.980714,28,19,19.0
1,10-14,96.95,2.770000,35,23,23.0
2,15-19,386.42,2.905414,133,100,100.0
3,20-24,978.77,2.913006,336,259,259.0
4,25-29,370.33,2.962640,125,87,87.0
5,30-34,197.25,3.082031,64,47,47.0
6,35-39,119.40,2.842857,42,27,27.0
7,40+,53.75,3.161765,17,11,11.0


In [25]:
#Top spenders
#Top 5 by total purchase value
#SN
#Purchase ct
#Avg purchase price
#Total purchase value
df.groupby('SN')['Price'].agg(['sum','mean','count']).sort_values(by='sum',ascending=False).head(5)

,sum,mean,count
SN,,,
Undirrala66,17.06,3.412000,5
Saedue76,13.56,3.390000,4
Mindimnya67,12.74,3.185000,4
Haellysu29,12.73,4.243333,3
Eoda93,11.58,3.860000,3


In [35]:
#Most Pop items
#the 5 most pop items by purchase ct, then list item id, item name, purch ct, item price total purch value
#items_counts = df['Item Name'].value_counts().reset_index()
#items_counts.head(5)
df.groupby('Item Name')['Price'].agg(['sum','mean','count']).sort_values(by='count',ascending=False).head(5)


,sum,mean,count
Item Name,,,
Final Critic,38.60,2.757143,14
Arcane Gem,24.53,2.230000,11
"Betrayal, Whisper of Grieving Widows",25.85,2.350000,11
Stormcaller,34.65,3.465000,10
Woeful Adamantite Claymore,11.16,1.240000,9


In [53]:
#Most Profitable Items
#Identify the 5 most profitable items by total purchase value
#Item Name
#Purchase ct
#Item price
#total purchase value
df.groupby('Item Name')['Price'].agg(['sum','mean','count']).sort_values(by='sum',ascending=False).head(5)




,sum,mean,count
Item Name,,,
Final Critic,38.60,2.757143,14
Retribution Axe,37.26,4.140000,9
Stormcaller,34.65,3.465000,10
Spectral Diamond Doomblade,29.75,4.250000,7
Orenmir,29.70,4.950000,6
